In [6]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
import numpy as np
import requests

In [8]:
x = requests.get('https://blocks.flashbots.net/v1/blocks') # just an example of loading in json via GET request

In [9]:
data = x.json()

In [10]:
with open('..\Ethdata\latest_blocks.json') as json_file: # how you would load in an already uploaded json file
    data = json.load(json_file)

In [11]:
data[0]

{'block_number': 14986955,
 'miner_reward': '358920921362317961',
 'miner': '0x646dB8ffC21e7ddc2B6327448dd9Fa560Df41087',
 'coinbase_transfers': '358920921362317961',
 'gas_used': 453908,
 'gas_price': '790734953696',
 'transactions': [{'transaction_hash': '0x7b2640a1915092f9c14a050f4a2ffcfa623396ffe3bfcde3a436c68f56487a35',
   'tx_index': 0,
   'bundle_type': 'flashbots',
   'bundle_index': 0,
   'block_number': 14986955,
   'eoa_address': '0xD295879063BE7A6c49F7327AfaA831a4f3f64882',
   'to_address': '0x4Cb18386e5d1F34dC6EEA834bf3534A970a3f8e7',
   'gas_used': 128763,
   'gas_price': '973175407353',
   'coinbase_transfer': '125308984977002215',
   'total_miner_reward': '125308984977002215'},
  {'transaction_hash': '0x8366ecc71dbe8ef7d487ce2628378ab86651ad7a2df99eb3e18bc6af9f469946',
   'tx_index': 0,
   'bundle_type': 'flashbots',
   'bundle_index': 1,
   'block_number': 14986955,
   'eoa_address': '0xeCa2e2D894D19778939bD4DfC34D2a3C45E96456',
   'to_address': '0x4Cb18386e5d1F34dC6EE

In [12]:
blockdf2 = pd.DataFrame(data)

In [13]:
blockdf2

,block_number,miner_reward,miner,coinbase_transfers,gas_used,gas_price,transactions
0,14986955,358920921362317961,0x646dB8ffC21e7ddc2B6327448dd9Fa560Df41087,358920921362317961,453908,790734953696,[{'transaction_hash': '0x7b2640a1915092f9c14a0...
1,14986954,266926641844315607,0x7F101fE45e6649A6fB8F3F8B43ed03D353f2B90c,233815223189565477,707043,377525329922,[{'transaction_hash': '0xae32bea44c40890ad2bee...
2,14986952,689259070093777007,0x646dB8ffC21e7ddc2B6327448dd9Fa560Df41087,154128719565454796,5304326,129942818388,[{'transaction_hash': '0xb97d89218f7924b1024a8...
3,14986949,381788848142916699,0x829BD824B016326A401d083B33D092293333A830,340862284848386550,702543,543438406108,[{'transaction_hash': '0x2d91fe1ae534143b2d3a4...
4,14986948,286956710025995738,0xEA674fdDe714fd979de3EdF0F56AA9716B898ec8,286803725093765114,600407,477936982790,[{'transaction_hash': '0x6cff42d45c83acddf7b6b...
...,...,...,...,...,...,...,...
1484616,11835001,26661835635854408,0xD224cA0c819e8E97ba0136B3b95ceFf503B79f53,26661835635854408,77137,345642631108,[{'transaction_hash': '0x29f4839ceea224bda3bbf...
1484617,11834918,32353817313917581,0xD224cA0c819e8E97ba0136B3b95ceFf503B79f53,32353817313917581,209853,154173718335,[{'transaction_hash': '0xe72d5fc908693361c4cbe...
1484618,11834911,227431738484650945,0xD224cA0c819e8E97ba0136B3b95ceFf503B79f53,227431738484650945,2696401,84346407854,[{'transaction_hash': '0xebbb811712c3560a96dc6...
1484619,11834380,7298671650726701,0xD224cA0c819e8E97ba0136B3b95ceFf503B79f53,7298671650726701,175813,41513833736,[{'transaction_hash': '0x03ebde03df94ec9883ef0...


In [14]:
gas_used = list(blockdf2.groupby('block_number')['gas_used'].sum())  # total gas used per block column

In [15]:
gas_price = list(blockdf2.groupby('block_number')['gas_price'].sum()) #total gas price per block column

In [16]:
final_df = pd.DataFrame(blockdf2.groupby('block_number')['miner_reward'].sum()) #quantifies raw MEV per block (in wei)

In [17]:
final_df['gas_used'] = gas_used        # adding columns into final df
final_df['gas_price'] = gas_price

In [18]:
final_df.head()  # propose using gas price as alternative to fluctuating eth price for time being, need to possibly query separate api with number of different oracles to obtain accurate eth price data per block

,miner_reward,gas_used,gas_price
block_number,,,
11834049,372250038378476828,4505938,82613218019
11834380,7298671650726701,175813,41513833736
11834911,227431738484650945,2696401,84346407854
11834918,32353817313917581,209853,154173718335
11835001,26661835635854408,77137,345642631108


In [19]:
def condition(x):
    if 0 <= x <= 4.32*(10**14):
         return 1
    elif 4.32*(10**14) < x <= 3.39*(10**15):
        return 2
    elif x > 3.39*(10**15):
        return 3

In [20]:
final_df['miner_reward'] = final_df['miner_reward'].astype('float') # change column data type so it will work with lambda condition

In [21]:
final_df.dtypes

miner_reward    float64
gas_used          int64
gas_price        object
dtype: object

In [22]:
final_df['MEV_category'] = final_df['miner_reward'].apply(condition) # new MEV category column

In [23]:
final_df.reset_index() #use this if you want to remove the remove the index - use 'inplace=True' as argument

,block_number,miner_reward,gas_used,gas_price,MEV_category
0,11834049,3.722500e+17,4505938,82613218019,3.0
1,11834380,7.298672e+15,175813,41513833736,3.0
2,11834911,2.274317e+17,2696401,84346407854,3.0
3,11834918,3.235382e+16,209853,154173718335,3.0
4,11835001,2.666184e+16,77137,345642631108,3.0
...,...,...,...,...,...
1484616,14986948,2.869567e+17,600407,477936982790,3.0
1484617,14986949,3.817888e+17,702543,543438406108,3.0
1484618,14986952,6.892591e+17,5304326,129942818388,3.0
1484619,14986954,2.669266e+17,707043,377525329922,3.0


In [25]:
final_df.to_csv('../Ethdata/temp_df.csv') # how you would convert final dataframe to csv

: 